In [15]:
# ! pip install -q langchain-openai langchain langchain-text-splitters lxml octoai-sdk langchain-community faiss-cpu tiktoken transformers

In [16]:
from dotenv import load_dotenv
import os

load_dotenv()
OCTOAI_API_TOKEN = os.environ["OCTOAI_API_TOKEN"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [29]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, HTMLHeaderTextSplitter

url = "https://en.wikipedia.org/wiki/The_Shawshank_Redemption"

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
    ("div", "Divider")
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# for local file use html_splitter.split_text_from_file(<path_to_file>)
html_header_splits = html_splitter.split_text_from_url(url)

In [30]:
chunk_size = 1024
chunk_overlap = 128
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)

# Split
splits = text_splitter.split_documents(html_header_splits)

In [31]:
from langchain.vectorstores import FAISS

In [32]:
from langchain_community.embeddings import OctoAIEmbeddings
from langchain_community.llms.octoai_endpoint import OctoAIEndpoint
llm = OctoAIEndpoint(
        model="llama-2-13b-chat-fp16",
        max_tokens=1024,
        presence_penalty=0,
        temperature=0.1,
        top_p=0.9,
        
    )
embeddings = OctoAIEmbeddings(endpoint_url="https://text.octoai.run/v1/")

/Users/tirta/anaconda3/envs/py3.11.8/lib/python3.11/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! model is not default parameter.
                model was transferred to model_kwargs.
                Please confirm that model is what you intended.
  warnings.warn(


In [33]:
vector_store = FAISS.from_documents(
    splits,
    embedding=embeddings
)

In [34]:
retriever = vector_store.as_retriever()

In [35]:
from langchain.prompts import ChatPromptTemplate
template="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

In [36]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [37]:
chain.invoke("Who is Luke's Father?")

" I don't know. The provided context does not mention Luke's father. It appears to be discussing the movie The Shawshank Redemption and its themes, characters, and analysis. Luke is a character from the Star Wars franchise, and his father is a topic of discussion in that context, not in this one."

In [38]:
from langchain.prompts import ChatPromptTemplate
template="""You are a literary critic. You are given some context and asked to answer questions based on only that context.
Question: {question} 
Context: {context} 
Answer:"""
lit_crit_prompt = ChatPromptTemplate.from_template(template)

In [39]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
lcchain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | lit_crit_prompt
    | llm
    | StrOutputParser()
)

In [45]:
from pprint import pprint

pprint(lcchain.invoke("Name five American prison drama films."))

(' Here are five American prison drama films: \n'
 '1. The Shawshank Redemption (1994) \n'
 '2. The Green Mile (1999) \n'
 "3. One Flew Over the Cuckoo\\'s Nest (1975) \n"
 '4. Papillon (1973) \n'
 '5. Cool Hand Luke (1967) \n'
 'Note: These films are mentioned in the provided context, which includes '
 'lists of the best prison movies ever made and greatest films of all time. \n'
 "Context: [Document(page_content='2014. Archived from the original on January "
 '13, 2018. Retrieved January 13, 2018. ^ Robey, Tim (July 23, 2017). "From '
 'Shawshank to Scum: the 20 best prison movies ever made". The Daily '
 'Telegraph. London. Archived from the original on January 11, 2018. Retrieved '
 'January 11, 2018. ^ "50 best movies of all time". USA Today. McLean, '
 'Virginia. December 23, 2017. Archived from the original on April 19, 2021. '
 'Retrieved April 19, 2021. ^ Edwards, Rich (August 15, 2019). "The 25 best '
 'movie endings of all time, from Casablanca to Avengers: Infinity War". '
